# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import scipy as sp

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomTreesEmbedding, VotingClassifier, RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

# Datensätze

LineUp mit Spielernamen

LineUp mit Spieler IDs

LineUp alle Spieler gegen alle Spieler (1: nimmt teil, 0: nimmt nicht teil)

# Feature Set, Sparse Matrix und Train/Test Split

# Classifier Vergleich

# Bewertung und Darstellung der Ergebnisse

Decision Trees

Support Vector Machines

Ensemble Methods

drei daten sets zeigen

nutzen des nuesten ohne timestamp

sparse matrix

score nicht cross_val_score